In [2]:
from sys import stdout
import parmed as pmd
# OpenMM imports
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from openmmforcefields.generators import SMIRNOFFTemplateGenerator
from openmm.app import PDBFile
# OpenFF-toolkit imports
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm

In [3]:
protein_path = "CYPA-processed.pdb"
ligand_path = "truncated_ligand_noH.sdf"
ligand = Molecule.from_file(ligand_path)
protein_pdb = app.PDBFile(protein_path)
smirnoff = SMIRNOFFTemplateGenerator(molecules=ligand, forcefield="openff-2.2.1.offxml")
ff = app.ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml')
ff.registerTemplateGenerator(smirnoff.generator)
#
modeller = app.Modeller(protein_pdb.topology, protein_pdb.positions)
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])
ligand_omm_topology = ligand_off_topology.to_openmm()
ligand_positions = offquantity_to_openmm(ligand.conformers[0])
modeller.add(ligand_omm_topology, ligand_positions)
modeller.addSolvent(ff, padding=1.0*unit.nanometer, ionicStrength=0.15*unit.molar)
#
system_1 = ff.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

In [4]:
with open('system.xml', 'w') as f:
    f.write(mm.XmlSerializer.serialize(system_1))
with open('complex_in_solvent.pdb', 'w') as output_pdb:
    PDBFile.writeFile(modeller.topology, modeller.positions, output_pdb)

In [5]:
system_1_gmx = ff.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=1.0*unit.nanometer, constraints=None, rigidWater=False)
structure = pmd.openmm.load_topology(modeller.topology, system=system_1_gmx, xyz=modeller.positions)
structure.save('gromacs.top', overwrite=True)
structure.save('gromacs.gro', overwrite=True)